In [7]:
# notebook setup
import datetime as dt
import os
import sys
from pathlib import Path
p = Path().resolve().parents[1]
sys.path.append(str(p))
from IPython.core.interactiveshell import InteractiveShell
import pandas as pd

import gscap.weather as weather

InteractiveShell.ast_node_interactivity = 'all'

### Get a daily weather report by latitude and longitude

In [9]:
now = dt.datetime.now()
today = dt.datetime(year=now.year, month=now.month, day=now.day); today

report = weather.weather_report((32.503082, -97.187339, today))
pd.DataFrame(report.get('report')).head()

datetime.datetime(2022, 2, 13, 0, 0)

,date,lat,lon,zipcode,cloud_cover_mean,cloud_cover_std,cloud_cover_median,cloud_cover_IQR,dew_point_mean,dew_point_std,...,dew_point_IQR,humidity_mean,humidity_std,humidity_median,humidity_IQR,precip_sum,temp_mean,temp_std,temp_med,temp_IQR
0,2022-02-13,32.503082,-97.187339,76061,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


If you notice above, report comes back from the weather module as a dictionary. It contains three attributes: hits, misses, and the report itself. `hits` tells you how many times you were able to retrieve a request from the cache while `misses` tells you how many API calls were made. This is extremely valuable for tracking expenses. You can view them as shown in the next cell.

In [10]:
report['hits']
report['misses']

0

1

### Or get a report by zipcode

In [11]:
report = weather.weather_report((today, 76092))
pd.DataFrame(report.get('report')).head()

,date,lat,lon,zipcode,cloud_cover_mean,cloud_cover_std,cloud_cover_median,cloud_cover_IQR,dew_point_mean,dew_point_std,...,dew_point_IQR,humidity_mean,humidity_std,humidity_median,humidity_IQR,precip_sum,temp_mean,temp_std,temp_med,temp_IQR
0,2022-02-13,32.95987,-97.14843,76092,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### If you have a series of requests to make, just include a list of a requests. You can set `n_jobs` to use as many cores as you want. 
For example, the following request makes four simultaneous calls to Dark Sky retrieving daily weather summaries by zipcode or lat-lon pairs, and DateTimes in any order. The only order that must be maintained is that the latitude must come before the longitude.

In [5]:
requests = [
    (today, 47579),
    (47579, today-dt.timedelta(days=1)),
    (38.11, -86.92, today-dt.timedelta(days=2)),
    (today-dt.timedelta(days=3), 38.11, -86.92),
    (38.11, today-dt.timedelta(days=4), -86.92),
]

report = weather.weather_report(requests)

pd.DataFrame(report.get('report')).head()
print(f'hits: {report["hits"]}, misses: {report["misses"]}')

,cloud_cover_IQR,cloud_cover_mean,cloud_cover_median,cloud_cover_std,date,dew_point_IQR,dew_point_mean,dew_point_median,dew_point_std,humidity_IQR,...,humidity_median,humidity_std,lat,lon,precip_sum,temp_IQR,temp_mean,temp_med,temp_std,zipcode
0,0.39,0.73,0.73,0.25,2019-01-24,3.18,20.68,21.21,3.07,0.13,...,0.72,0.07,38.11094,-86.91513,0.01,6.72,28.48,27.35,4.25,47579
1,0.00,1.00,1.00,0.00,2019-01-23,13.77,39.69,42.39,8.18,0.06,...,0.93,0.06,38.11094,-86.91513,0.66,15.39,41.78,47.09,8.47,47579
2,1.00,0.51,0.65,0.44,2019-01-22,17.56,22.71,23.99,9.61,0.11,...,0.61,0.07,38.11000,-86.92000,0.00,22.22,35.81,41.89,10.66,47579
3,0.00,0.09,0.00,0.22,2019-01-21,0.89,9.25,9.41,0.89,0.32,...,0.62,0.16,38.11000,-86.92000,0.00,10.88,18.36,21.64,5.98,47579
4,0.52,0.71,0.78,0.33,2019-01-20,8.39,12.53,12.36,4.89,0.26,...,0.74,0.13,38.11000,-86.92000,0.00,2.55,21.40,21.97,2.54,47579


hits: 0, misses: 5


### The default summarization is at the daily level as shown above. To return the results at they hourly level set `summarize = 'none'`

In [6]:
report = weather.weather_report(requests, summarize='none')
report = pd.DataFrame(report.get('report'))
report.head(50)

,apparentTemperature,cloudCover,cloudCoverError,date,dewPoint,hour,humidity,icon,lat,lon,...,temperatureError,time,uvIndex,visibility,windBearing,windBearingError,windGust,windSpeed,windSpeedError,zipcode
0,16.58,1.00,None,2019-01-24,21.52,0,0.84,cloudy,38.11094,-86.91513,...,None,00:00:00,0.0,10.00,296.0,None,10.91,9.04,None,47579
1,15.33,1.00,None,2019-01-24,20.05,1,0.80,cloudy,38.11094,-86.91513,...,None,01:00:00,0.0,10.00,300.0,None,10.91,10.28,None,47579
2,15.14,1.00,None,2019-01-24,19.59,2,0.80,cloudy,38.11094,-86.91513,...,None,02:00:00,0.0,10.00,295.0,None,9.80,9.76,None,47579
3,15.96,1.00,None,2019-01-24,19.47,3,0.80,cloudy,38.11094,-86.91513,...,None,03:00:00,0.0,10.00,285.0,None,8.21,8.04,None,47579
4,14.86,1.00,None,2019-01-24,18.96,4,0.79,cloudy,38.11094,-86.91513,...,None,04:00:00,0.0,9.51,290.0,None,9.21,9.21,None,47579
5,12.38,1.00,None,2019-01-24,17.80,5,0.79,cloudy,38.11094,-86.91513,...,None,05:00:00,0.0,10.00,286.0,None,11.67,11.14,None,47579
6,15.12,0.48,None,2019-01-24,17.74,6,0.76,partly-cloudy-day,38.11094,-86.91513,...,None,06:00:00,0.0,10.00,290.0,None,8.11,8.11,None,47579
7,16.21,0.06,None,2019-01-24,19.29,7,0.79,clear-day,38.11094,-86.91513,...,None,07:00:00,1.0,10.00,270.0,None,8.37,8.17,None,47579
8,20.78,0.22,None,2019-01-24,19.70,8,0.73,clear-day,38.11094,-86.91513,...,None,08:00:00,2.0,10.00,252.0,None,8.05,5.89,None,47579
9,21.73,0.71,None,2019-01-24,21.21,9,0.72,partly-cloudy-day,38.11094,-86.91513,...,None,09:00:00,2.0,10.00,263.0,None,8.01,7.42,None,47579
